In [1]:
import os
import sys

sys.path.append(os.path.dirname(os.getcwd())) 
import logging 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.stats.outliers_influence import variance_inflation_factor
import numpy as np

from src.features.load_games_from_db import load_games_from_db

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
model_data_path = '../data/processed/model_data.csv'
df = pd.read_csv(model_data_path)

In [4]:
df.head()

,game_id,game_date,game_date_time,home_team_id,away_team_id,home_team,away_team,home_score,away_score,state,venue,game_type,home_team_rest_days,home_team_games_prev_7days,home_team_season_opener_flag,away_team_rest_days,away_team_games_prev_7days,away_team_season_opener_flag,home_back2back,away_back2back,rest_difference,home_win_rate_last_10,home_avg_run_diff_last_10,home_std_run_diff_last_10,home_avg_runs_scored_last_10,home_std_runs_scored_last_10,home_avg_runs_allowed_last_10,home_std_runs_allowed_last_10,away_win_rate_last_10,away_avg_run_diff_last_10,away_std_run_diff_last_10,away_avg_runs_scored_last_10,away_std_runs_scored_last_10,away_avg_runs_allowed_last_10,away_std_runs_allowed_last_10,game_month,game_day_of_week,game_hour,home_rolling_avg_obp_7days,home_rolling_avg_obp_14days,home_rolling_avg_slg_7days,home_rolling_avg_slg_14days,home_rolling_avg_era_7days,home_rolling_avg_era_14days,home_rolling_avg_whip_7days,home_rolling_avg_whip_14days,home_rolling_avg_strikeouts_pitching_7days,home_rolling_avg_strikeouts_pitching_14days,home_rolling_avg_baseonballs_pitching_7days,home_rolling_avg_baseonballs_pitching_14days,home_rolling_avg_hits_pitching_7days,home_rolling_avg_hits_pitching_14days,home_rolling_std_obp_7days,home_rolling_std_obp_14days,home_rolling_std_slg_7days,home_rolling_std_slg_14days,home_rolling_std_era_7days,home_rolling_std_era_14days,home_rolling_std_whip_7days,home_rolling_std_whip_14days,home_rolling_std_strikeouts_pitching_7days,home_rolling_std_strikeouts_pitching_14days,home_rolling_std_baseonballs_pitching_7days,home_rolling_std_baseonballs_pitching_14days,home_rolling_std_hits_pitching_7days,home_rolling_std_hits_pitching_14days,home_obp_lag1,home_obp_lag2,home_obp_lag3,home_slg_lag1,home_slg_lag2,home_slg_lag3,home_era_lag1,home_era_lag2,home_era_lag3,home_whip_lag1,home_whip_lag2,home_whip_lag3,home_strikeouts_pitching_lag1,home_strikeouts_pitching_lag2,home_strikeouts_pitching_lag3,home_baseonballs_pitching_lag1,home_baseonballs_pitching_lag2,home_baseonballs_pitching_lag3,home_hits_pitching_lag1,home_hits_pitching_lag2,home_hits_pitching_lag3,away_rolling_avg_obp_7days,away_rolling_avg_obp_14days,away_rolling_avg_slg_7days,away_rolling_avg_slg_14days,away_rolling_avg_era_7days,away_rolling_avg_era_14days,away_rolling_avg_whip_7days,away_rolling_avg_whip_14days,away_rolling_avg_strikeouts_pitching_7days,away_rolling_avg_strikeouts_pitching_14days,away_rolling_avg_baseonballs_pitching_7days,away_rolling_avg_baseonballs_pitching_14days,away_rolling_avg_hits_pitching_7days,away_rolling_avg_hits_pitching_14days,away_rolling_std_obp_7days,away_rolling_std_obp_14days,away_rolling_std_slg_7days,away_rolling_std_slg_14days,away_rolling_std_era_7days,away_rolling_std_era_14days,away_rolling_std_whip_7days,away_rolling_std_whip_14days,away_rolling_std_strikeouts_pitching_7days,away_rolling_std_strikeouts_pitching_14days,away_rolling_std_baseonballs_pitching_7days,away_rolling_std_baseonballs_pitching_14days,away_rolling_std_hits_pitching_7days,away_rolling_std_hits_pitching_14days,away_obp_lag1,away_obp_lag2,away_obp_lag3,away_slg_lag1,away_slg_lag2,away_slg_lag3,away_era_lag1,away_era_lag2,away_era_lag3,away_whip_lag1,away_whip_lag2,away_whip_lag3,away_strikeouts_pitching_lag1,away_strikeouts_pitching_lag2,away_strikeouts_pitching_lag3,away_baseonballs_pitching_lag1,away_baseonballs_pitching_lag2,away_baseonballs_pitching_lag3,away_hits_pitching_lag1,away_hits_pitching_lag2,away_hits_pitching_lag3,rolling_avg_obp_7days_diff,rolling_avg_obp_7days_ratio,rolling_avg_obp_14days_diff,rolling_avg_obp_14days_ratio,rolling_avg_slg_7days_diff,rolling_avg_slg_7days_ratio,rolling_avg_slg_14days_diff,rolling_avg_slg_14days_ratio,rolling_avg_era_7days_diff,rolling_avg_era_7days_ratio,rolling_avg_era_14days_diff,rolling_avg_era_14days_ratio,rolling_avg_whip_7days_diff,rolling_avg_whip_7days_ratio,rolling_avg_whip_14days_diff,rolling_avg_whip_14days_ratio,rolling_avg_strikeouts_pitching_7days_diff,rolling_avg_strik

In [5]:
df.shape

(10997, 193)

In [6]:
df['state'].value_counts(dropna=False)

state
Final              10955
Completed Early       28
Scheduled             14
Name: count, dtype: int64

In [7]:
df = df[df['state']!='Scheduled']

In [8]:
df['game_type'].value_counts(dropna=False)

game_type
R    10983
Name: count, dtype: int64

In [9]:
df.select_dtypes(include='object').head()

,game_date,game_date_time,home_team,away_team,state,venue,game_type
0,2021-04-01,2021-04-01 17:05:00+00:00,New York Yankees,Toronto Blue Jays,Final,Yankee Stadium,R
1,2021-04-01,2021-04-01 17:10:00+00:00,Detroit Tigers,Cleveland Guardians,Final,Comerica Park,R
2,2021-04-01,2021-04-01 18:10:00+00:00,Milwaukee Brewers,Minnesota Twins,Final,American Family Field,R
3,2021-04-01,2021-04-01 18:20:00+00:00,Chicago Cubs,Pittsburgh Pirates,Final,Wrigley Field,R
4,2021-04-01,2021-04-01 19:05:00+00:00,Philadelphia Phillies,Atlanta Braves,Final,Citizens Bank Park,R


In [13]:
target = 'home_win'

cols_to_remove = [target, 'game_id','game_date','game_date_time','home_team_id','away_team_id', 'state', 'game_type', 'home_score','away_score']
cat_cols = ['home_team','away_team','venue']

cutoff_date = '2025-05-15'

num_cols = list((set(df.columns)-set(cols_to_remove))-set(cat_cols))

In [15]:
cat_cols

['home_team', 'away_team', 'venue']

In [11]:
df_train = df[df['game_date']<='2025-05-15'].copy()

In [12]:
df_train.shape

(10166, 193)

In [90]:
X_num.describe()

,rolling_avg_obp_7days_diff,home_std_runs_allowed_last_10,rolling_avg_strikeouts_pitching_14days_ratio,away_rolling_avg_era_14days,home_strikeouts_pitching_lag2,rolling_avg_slg_14days_ratio,away_rolling_avg_obp_7days,home_strikeouts_pitching_lag3,home_baseonballs_pitching_lag3,home_rolling_avg_hits_pitching_14days,home_hits_pitching_lag2,home_rolling_std_slg_14days,rolling_avg_baseonballs_pitching_7days_ratio,home_rolling_std_era_14days,rolling_std_obp_7days_diff,game_day_of_week,home_rolling_std_obp_7days,away_whip_lag3,rolling_avg_slg_7days_diff,away_baseonballs_pitching_lag2,away_strikeouts_pitching_lag3,rolling_avg_hits_pitching_14days_ratio,rolling_std_obp_14days_ratio,away_strikeouts_pitching_lag2,home_rolling_std_hits_pitching_7days,rolling_std_hits_pitching_7days_ratio,home_rolling_std_whip_7days,rolling_avg_strikeouts_pitching_7days_diff,home_rolling_avg_whip_7days,away_rolling_std_obp_14days,away_slg_lag1,rolling_std_era_14days_diff,rolling_avg_whip_14days_ratio,away_win_rate_last_10,away_team_rest_days,away_rolling_avg_slg_14days,home_obp_lag2,rolling_avg_slg_7days_ratio,away_rolling_std_baseonballs_pitching_7days,away_rolling_avg_era_7days,home_avg_runs_allowed_last_10,away_rolling_avg_baseonballs_pitching_14days,away_rolling_std_hits_pitching_14days,rolling_std_obp_14days_diff,home_rolling_avg_obp_14days,rolling_std_whip_7days_ratio,away_rolling_avg_hits_pitching_14days,rolling_avg_whip_7days_ratio,home_team_games_prev_7days,rolling_std_baseonballs_pitching_14days_diff,home_obp_lag3,away_baseonballs_pitching_lag1,rolling_avg_era_7days_diff,home_rolling_avg_baseonballs_pitching_14days,game_month,away_rolling_std_whip_14days,home_obp_lag1,home_rolling_avg_baseonballs_pitching_7days,away_obp_lag2,away_rolling_std_strikeouts_pitching_14days,home_rolling_avg_whip_14days,rolling_std_whip_7days_diff,rest_difference,home_baseonballs_pitching_lag2,home_rolling_avg_hits_pitching_7days,rolling_avg_era_14days_ratio,home_baseonballs_pitching_lag1,away_avg_runs_scored_last_10,home_whip_lag2,rolling_avg_baseonballs_pitching_14days_ratio,rolling_avg_era_7days_ratio,rolling_avg_era_14days_diff,away_slg_lag3,rolling_std_baseonballs_pitching_14days_ratio,rolling_avg_baseonballs_pitching_7days_diff,away_rolling_std_hits_pitching_7days,away_rolling_std_era_14days,home_rolling_std_baseonballs_pitching_14days,home_whip_lag1,rolling_avg_baseonballs_pitching_14days_diff,rolling_std_hits_pitching_14days_diff,away_std_runs_scored_last_10,away_baseonballs_pitching_lag3,rolling_std_hits_pitching_14days_ratio,away_std_runs_allowed_last_10,home_hits_pitching_lag1,rolling_avg_hits_pitching_14days_diff,home_rolling_std_strikeouts_pitching_14days,away_rolling_avg_strikeouts_pitching_14days,rolling_avg_obp_14days_ratio,home_strikeouts_pitching_lag1,away_avg_run_diff_last_10,rolling_std_baseonballs_pitching_7days_diff,rolling_std_strikeouts_pitching_14days_ratio,home_team_rest_days,rolling_std_slg_7days_ratio,away_rolling_avg_whip_14days,rolling_std_slg_14days_diff,home_rolling_avg_slg_7days,rolling_avg_strikeouts_pitching_14days_diff,home_rolling_std_obp_14days,home_rolling_std_hits_pitching_14days,away_rolling_std_whip_7days,away_era_lag1,home_rolling_avg_strikeouts_pitching_7days,home_rolling_std_whip_14days,home_std_runs_scored_last_10,rolling_std_whip_14days_diff,away_team_season_opener_flag,away_std_run_diff_last_10,away_whip_lag1,away_rolling_avg_obp_14days,home_rolling_avg_era_14days,away_rolling_avg_hits_pitching_7days,rolling_std_strikeouts_pitching_7days_ratio,home_back2back,rolling_std_slg_14days_ratio,away_whip_lag2,home_slg_lag3,rolling_std_baseonballs_pitching_7days_ratio,away_rolling_std_slg_7days,rolling_avg_strikeouts_pitching_7days_ratio,home_rolling_avg_strikeouts_pitching_14days,rolling_std_strikeouts_pitching_7days_diff,rolling_avg_slg_14days_diff,home_rolling_std_strikeouts_pitching_7days,away_era_lag2,home_win_rate_last_10,home_rolling_std_era_7days,away_rolling_std_slg_14days,away_hits_pitching_lag3,away_rolling_avg_baseonba

In [16]:
X_num = df_train[num_cols].copy()

X_cat = df_train[cat_cols].copy()


In [17]:
X_num.shape, X_cat.shape

((10166, 180), (10166, 3))

In [18]:
cormat = X_num.corr()

In [27]:
vif_data = pd.DataFrame()
vif_data["feature"] = X_num.columns

vif_data["VIF"] = [variance_inflation_factor(X_num.values, i)
                          for i in range(len(X_num.columns))]


/Users/samuelluxenberg/Library/CloudStorage/OneDrive-Personal/GitHub/end-to-end-mlb-betting/.venv/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


In [44]:
vif_data["R-sq"] = 1/(1-vif_data['VIF'])

In [48]:
vif_data.sort_values('VIF')

,feature,VIF,R-sq
15,game_day_of_week,3.863842e+00,-3.491812e-01
156,game_hour,4.058045e+00,-3.270063e-01
8,home_baseonballs_pitching_lag3,4.683410e+00,-2.714876e-01
82,away_baseonballs_pitching_lag3,4.770916e+00,-2.651875e-01
63,home_baseonballs_pitching_lag2,4.779714e+00,-2.645702e-01
66,home_baseonballs_pitching_lag1,4.858269e+00,-2.591836e-01
19,away_baseonballs_pitching_lag2,4.937168e+00,-2.539897e-01
51,away_baseonballs_pitching_lag1,5.002577e+00,-2.498391e-01
130,away_hits_pitching_lag3,9.697598e+00,-1.149743e-01
163,home_hits_pitching_lag3,9.963576e+00,-1.115626e-01


In [45]:
vif_data[vif_data['VIF']<10]

,feature,VIF,R-sq
8,home_baseonballs_pitching_lag3,4.683410,-0.271488
15,game_day_of_week,3.863842,-0.349181
19,away_baseonballs_pitching_lag2,4.937168,-0.253990
51,away_baseonballs_pitching_lag1,5.002577,-0.249839
63,home_baseonballs_pitching_lag2,4.779714,-0.264570
66,home_baseonballs_pitching_lag1,4.858269,-0.259184
82,away_baseonballs_pitching_lag3,4.770916,-0.265188
130,away_hits_pitching_lag3,9.697598,-0.114974
156,game_hour,4.058045,-0.327006
163,home_hits_pitching_lag3,9.963576,-0.111563


In [43]:
# Low variance?
X_num.std(axis=0).sort_values()

rolling_std_obp_7days_diff                       6.407457e-03
rolling_std_obp_14days_diff                      7.677404e-03
away_rolling_std_obp_7days                       7.733751e-03
home_rolling_std_obp_7days                       7.894799e-03
away_rolling_std_obp_14days                      9.578147e-03
home_rolling_std_obp_14days                      9.821915e-03
rolling_std_slg_7days_diff                       1.207220e-02
away_rolling_std_slg_7days                       1.405612e-02
rolling_std_slg_14days_diff                      1.458339e-02
home_rolling_std_slg_7days                       1.509381e-02
away_rolling_std_slg_14days                      1.775355e-02
home_rolling_std_slg_14days                      1.840558e-02
rolling_avg_obp_7days_diff                       3.053673e-02
rolling_avg_obp_14days_diff                      3.159601e-02
home_obp_lag1                                    3.302752e-02
away_obp_lag1                                    3.366454e-02
home_rol

In [49]:
from sklearn.preprocessing import StandardScaler  # to standardize the features
from sklearn.decomposition import PCA  # to apply PCA

In [51]:
scalar = StandardScaler() 
scaled_data = pd.DataFrame(scalar.fit_transform(X_num)) #scaling the data
scaled_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179
0,0.007353,-2.988696,-0.019449,-4.258,-2.745289,-0.019752,-9.030087,-2.62892,-1.506742,-6.091478,-2.297474,-0.547663,-0.018916,-0.514509,-0.009348,-0.111083,-0.456756,-5.443752,0.013839,-1.55468,-2.629915,-0.018342,-0.020402,-2.682757,-2.801126,-0.039217,-0.446328,0.006872,-7.218822,-0.553722,-7.624737,0.00056,-0.0182,-2.671,-0.343323,-7.251019,-7.328637,-0.019752,-2.79899,-4.364939,-3.633636,-4.188533,-3.656964,-0.008515,-9.100221,-0.051533,-6.132274,-0.0182,-4.589585,-0.005062,-6.161232,-1.575633,-0.005632,-4.100314,-1.327505,-0.55069,-9.449641,-3.394134,-7.178435,-3.942213,-7.093001,-0.023354,-0.021685,-1.517593,-5.106026,-0.024767,-1.537308,-3.680915,-6.248409,-0.018916,-0.024767,0.004904,-5.519426,-0.038134,0.001427,-2.808407,-0.528379,-3.610516,-7.458203,-0.000988,-0.002333,-2.95959,-1.525433,-0.039217,-3.003558,-2.337259,-0.013722,-3.900896,-6.390416,-0.019767,-2.886897,-0.00975,-0.013264,-0.032604,-0.349241,-0.027942,-7.084526,-0.006951,-7.484844,-0.004798,-0.546635,-3.621516,-0.467722,-4.580312,-5.441526,-0.535816,-2.959266,-0.014813,11.52229,-3.251481,-7.408721,-8.919399,-4.396931,-5.131653,-0.032604,-2.355898,-0.027867,-6.214616,-5.580981,-0.038134,-0.472104,-0.019449,-6.348216,-0.008742,0.005304,-3.011202,-4.267821,-2.618435,-0.4232,-0.562067,-2.267943,-3.455447,-6.32297,-4.025783,-9.265637,-7.690804,-7.371917,-0.458522,-2.732589,-0.019767,-2.349125,-5.512461,-6.40291,-3.018593,-0.026129,-3.209776,-0.015128,-7.191857,11.678115,0.007318,-0.023658,-5.479751,-2.422412,-0.452294,-7.339554,-3.635013,0.200471,-9.234578,-4.355242,-4.510919,-0.018302,-0.023658,-0.001309,-2.269958,-6.075794,-0.018342,-4.766104,0.007543,-4.013461,-0.024106,-4.585685,-2.399856,0.011024,-0.443998,-0.015812,-3.620939,-0.004327,-0.029875,-3.5672,-2.763954
1,0.007353,-2.988696,-0.019449,-4.258,-2.745289,-0.019752,-9.030087,-2.62892,-1.506742,-6.091478,-2.297474,-0.547663,-0.018916,-0.514509,-0.009348,-0.111083,-0.456756,-5.443752,0.013839,-1.55468,-2.629915,-0.018342,-0.020402,-2.682757,-2.801126,-0.039217,-0.446328,0.006872,-7.218822,-0.553722,-7.624737,0.00056,-0.0182,-2.671,-0.343323,-7.251019,-7.328637,-0.019752,-2.79899,-4.364939,-3.633636,-4.188533,-3.656964,-0.008515,-9.100221,-0.051533,-6.132274,-0.0182,-4.589585,-0.005062,-6.161232,-1.575633,-0.005632,-4.100314,-1.327505,-0.55069,-9.449641,-3.394134,-7.178435,-3.942213,-7.093001,-0.023354,-0.021685,-1.517593,-5.106026,-0.024767,-1.537308,-3.680915,-6.248409,-0.018916,-0.024767,0.004904,-5.519426,-0.038134,0.001427,-2.808407,-0.528379,-3.610516,-7.458203,-0.000988,-0.002333,-2.95959,-1.525433,-0.039217,-3.003558,-2.337259,-0.013722,-3.900896,-6.390416,-0.019767,-2.886897,-0.00975,-0.013264,-0.032604,-0.349241,-0.027942,-7.084526,-0.006951,-7.484844,-0.004798,-0.546635,-3.621516,-0.467722,-4.580312,-5.441526,-0.535816,-2.959266,-0.014813,11.52229,-3.251481,-7.408721,-8.919399,-4.396931,-5.131653,-0.032604,-2.355898,-0.027867,-6.214616,-5.580981,-0.038134,-0.472104,-0.019449,-6.348216,-0.008742,0.005304,-3.011202,-4.267821,-2.618435,-0.4232,-0.562067,-2.267943,-3.455447,-6.32297,-4.025783,-9.265637,-7.690804,-7.371917,-0.458522,-2.732589,-0.019767,-2.349125,-5.512461,-6.40291,-3.018593,-0.026129,-3.209776,-0.015128,-7.191857,11.678115,0.007318,-0.023658,-5.479751,-2.422412,-0.452294,-7.339554,-3.635013,0.200471,-9.234578,-4.355242,-4.510919,-0.018302,-0.023658,-0.001309,-2.269958,-6.075794,-

In [98]:

scaled_data.inf()

AttributeError: 'DataFrame' object has no attribute 'inf'

In [66]:
pca = PCA(n_components=.9)
my_pca = pca.fit(scaled_data)
#data_pca = pca.transform(scaled_data)
#data_pca = pd.DataFrame(data_pca,columns=['PC1','PC2','PC3'])
#data_pca.head()

#print(f"Explained Variance: {my_pca.explained_variance_}")
#print(f"Explained Variance Ratio: {my_pca.explained_variance_ratio_}")
print(f"Explained Variance Ratio CumSum: {my_pca.explained_variance_ratio_.cumsum()}") 

Explained Variance Ratio CumSum: [0.15670994 0.26370133 0.3526322  0.42045698 0.45811291 0.48977199
 0.51930277 0.54396336 0.56788458 0.58870099 0.60860676 0.62739828
 0.64538742 0.66029462 0.67440442 0.68766007 0.69979527 0.7110043
 0.72196701 0.73242365 0.74213185 0.75074478 0.75915308 0.7674502
 0.77514065 0.7826507  0.78997573 0.79686652 0.80341343 0.80980077
 0.81604811 0.8220953  0.82798114 0.83376951 0.8395045  0.84501953
 0.85030255 0.85557101 0.86052732 0.86537727 0.87018549 0.87491836
 0.87960526 0.88419754 0.88867852 0.89302095 0.89732819 0.90150445]


# Categorical Features

In [67]:
X_cat.head()

,home_team,away_team,venue
0,New York Yankees,Toronto Blue Jays,Yankee Stadium
1,Detroit Tigers,Cleveland Guardians,Comerica Park
2,Milwaukee Brewers,Minnesota Twins,American Family Field
3,Chicago Cubs,Pittsburgh Pirates,Wrigley Field
4,Philadelphia Phillies,Atlanta Braves,Citizens Bank Park


In [87]:
controls = cat_cols + ['game_day_of_week','game_hour', 'game_month']

In [69]:
controls

['home_team', 'away_team', 'venue', 'game_day_of_week', 'game_hour']

In [88]:
pitch_cols = list(df.columns[df.columns.str.contains('pitch|era|whip')])
run_cols = list(df.columns[df.columns.str.contains('run')])
hit_cols = list(df.columns[df.columns.str.contains('obp|slg')])
sched_cols = list(df.columns[df.columns.str.contains('games|rest|back2back|season')])
win_cols = list(df.columns[df.columns.str.contains('win_rate')])

In [89]:
scalar_pitch = StandardScaler() 
scaled_data = pd.DataFrame(scalar.fit_transform(X_num)) #scaling the data
pca_pitch = PCA(n_components=.9)
pitch_cols_pca = pca_pitch.fit(scaled_data)
print(f"Explained Variance Ratio CumSum: {pitch_cols_pca.explained_variance_ratio_.cumsum()}") 

Explained Variance Ratio CumSum: [0.15670994 0.26370133 0.3526322  0.42045698 0.45811291 0.48977199
 0.51930277 0.54396336 0.56788458 0.58870099 0.60860676 0.62739828
 0.64538742 0.66029462 0.67440442 0.68766007 0.69979527 0.7110043
 0.72196701 0.73242365 0.74213185 0.75074478 0.75915308 0.7674502
 0.77514065 0.7826507  0.78997573 0.79686652 0.80341343 0.80980077
 0.81604811 0.8220953  0.82798114 0.83376951 0.8395045  0.84501953
 0.85030255 0.85557101 0.86052732 0.86537727 0.87018549 0.87491836
 0.87960526 0.88419754 0.88867852 0.89302095 0.89732819 0.90150445]


In [95]:
d1 = pd.DataFrame({'x1':['a','b','c'],'x2':[1,2,3]})
d2 = pd.DataFrame({'x3':['d','e','f'], 'x4':[4,5,6]})
pd.concat([d1,d2],axis=1)

,x1,x2,x3,x4
0,a,1,d,4
1,b,2,e,5
2,c,3,f,6


In [80]:
list(win_cols)

['home_win_rate_last_10', 'away_win_rate_last_10']

In [86]:
set(df_train.columns)-set(pitch_cols + run_cols + hit_cols + era_cols + whip_cols + sched_cols + win_cols + controls)

{'away_score',
 'away_team_id',
 'game_date',
 'game_date_time',
 'game_id',
 'game_month',
 'game_type',
 'home_score',
 'home_team_id',
 'home_win',
 'state'}